In [ ]:
##CREATE CHATBOT : Projects using Groq

In [16]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [17]:
!conda install -c pytorch faiss-gpu -y

Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [18]:
!pip install groq
!pip install faiss-cpu

In [19]:
from groq import Groq
import pandas as pd
from sentence_transformers import SentenceTransformer
import json
import faiss
import numpy as np

In [20]:
client = Groq(api_key="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")
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

In [21]:
def load_csv(filepath):
  df= pd.read_csv(filepath)
  return df

In [22]:
df =load_csv("/content/Jio_Telecom_Churn.csv")
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,Payment Mode
0,7590-VHVEG,Female,0,Yes,No,1,Jio Postpaid,JioFiber Only,JioFiber,No,...,No,No,Not Subscribed,Not Subscribed,Pay-as-you-go,Yes,29.85,29.85,No,UPI
1,5575-GNVDE,Male,0,No,No,34,Jio Prepaid,Single Jio SIM,JioFiber,Yes,...,Yes,No,Not Subscribed,Not Subscribed,Jio Prime (1 Year),No,56.95,1889.5,No,UPI
2,3668-QPYBK,Male,0,No,No,2,Jio Prepaid,Single Jio SIM,JioFiber,Yes,...,No,No,Not Subscribed,Not Subscribed,Pay-as-you-go,Yes,53.85,108.15,Yes,UPI
3,7795-CFOCW,Male,0,No,No,45,Jio Postpaid,JioFiber Only,JioFiber,Yes,...,Yes,Yes,Not Subscribed,Not Subscribed,Jio Prime (1 Year),No,42.30,1840.75,No,Bank transfer (automatic)
4,9237-HQITU,Female,0,No,No,2,Jio Prepaid,Single Jio SIM,Jio True 5G,No,...,No,No,Not Subscribed,Not Subscribed,Pay-as-you-go,Yes,70.70,151.65,Yes,UPI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Jio Prepaid,Jio Dual SIM,JioFiber,Yes,...,Yes,Yes,JioTV,JioCinema,Jio Prime (1 Year),Yes,84.80,1990.5,No,UPI
7039,2234-XADUH,Female,0,Yes,Yes,72,Jio Prepaid,Jio Dual SIM,Jio True 5G,No,...,Yes,No,JioTV,JioCinema,Jio Prime (1 Year),Yes,103.20,7362.9,No,Credit card (automatic)
7040,4801-JZAZL,Female,0,Yes,Yes,11,Jio Postpaid,JioFiber Only,JioFiber,Yes,...,No,No,Not Subscribed,Not Subscribed,Pay-as-you-go,Yes,29.60,346.45,No,UPI
7041,8361-LTMKD,Male,1,Yes,No,4,Jio Prepaid,Jio Dual SIM,Jio True 5G,No,...,No,No,Not Subscribed,Not Subscribed,Pay-as-you-go,Yes,74.40,306.6,Yes,UPI


In [23]:
def chunk_dataframe(df,chunk_size):
  chunks = []
  for i in range(0,len(df), chunk_size):
    chunk = df.iloc[i:i+chunk_size]
    chunks.append(chunk.to_json())
  return chunks

In [24]:
chunks = chunk_dataframe(df,5)

In [25]:
### Now Emdedding it and saving it to Vector DB and Retrieve
def create_faiss_index(chunks):
  embeddings = embed_model.encode(chunks)
  dim = embeddings.shape[1]
  index = faiss.IndexFlatL2(dim)
  index.add(np.array(embeddings))
  return index,chunks,embeddings


In [26]:
index,chunks,embeddings = create_faiss_index(chunks)

In [27]:
embeddings

array([[-0.044769  ,  0.08998064,  0.0134927 , ..., -0.00899047,
         0.0276388 ,  0.0034674 ],
       [-0.03888929,  0.09290603,  0.00735654, ..., -0.02842479,
         0.01608168,  0.02994474],
       [-0.04536542,  0.09285755,  0.01687848, ..., -0.02049794,
         0.00307637,  0.01051082],
       ...,
       [-0.04339305,  0.0994866 ,  0.00896991, ..., -0.01498477,
         0.01315465,  0.02318855],
       [-0.03410904,  0.10486729,  0.0064215 , ..., -0.01692636,
         0.01875523,  0.02847383],
       [-0.0567509 ,  0.11966264,  0.02462366, ..., -0.0169966 ,
         0.00357054,  0.02280437]], dtype=float32)

In [28]:
## PerformRetrieval
def retrieve_relevant_chunks(query,index,chunks,embeddings,k =2):
  query_vec = embed_model.encode([query])
  distances,indices = index.search(np.array(query_vec),k)
  return [chunks[i] for i in indices[0]]

In [37]:
def generate_response_from_groq(context,query):
  types = df.to_string()
  prompt = f"""

   You are a smart data asssitant , here is some context from the csv file :
   {context}\n\n
    Question:
    {query}\n
    Answer:

    """
  completion = client.chat.completions.create(
        model="llama3-8b-8192", # llama3-8b-8192 # gpt-3.5-turbo
        messages=[
            {"role": "system", "content":"you are the data analyst"},
            {"role":"user","content":prompt}
        ],
        temperature = 0.2

        )

  return completion.choices[0].message.content

In [38]:
def generate_insight(context,query):
  stats = df.describe(include="all").to_string()
  prompt = f"""
  So here is the dataset Summary : \n{stats} \n\n
   Generate 3 smart insights  or trend  that you observed dataset :

    """
  completion = client.chat.completions.create(
        model="llama3-8b-8192", # llama3-8b-8192 # gpt-3.5-turbo
        messages= [
            {"role": "system", "content":"you are the data analyst"},
            {"role":"user","content":prompt}
        ],
        temperature = 0.2

        )

  return completion.choices[0].message.content

In [40]:
## Now we call the function which we created
if __name__ == "__main__":
  filepath = "/content/Jio_Telecom_Churn.csv"
  df = load_csv(filepath)
  chunks = chunk_dataframe(df,5)
  index,chunks,embeddings = create_faiss_index(chunks)

  while True:
    query = input("Enter your query :")
    if query.lower() == [["exit","quit"]]:
      print("Thanks for using the Bot")
      break
    elif query.lower() == "insights":
      print("Generating Smart insights : ")
      print(generate_insight(df))
    else:
      context = "\n\ n".join(retrieve_relevant_chunks(query,index,chunks,embeddings))
      response = generate_response_from_groq(context,query)
      print("Answer : " ,response)



Enter your query :exit
Answer :  It seems like you want me to exit the conversation. I'll do that. It was nice chatting with you, but I'm happy to wrap things up. If you need any help or have any questions in the future, feel free to reach out to me anytime. Have a great day!
Enter your query :quit
Answer :  It seems like you're asking me to analyze the provided CSV file. Before we begin, I'd like to clarify a few things. Are you looking for specific insights or would you like me to provide an overall summary of the data? Additionally, are there any specific columns or variables you'd like me to focus on?


KeyboardInterrupt: Interrupted by user